## Preparing the workspace

In [5]:
#r "netstandard"
#load "Paket.fsx"

In [7]:
Paket.Dependencies.Install """
storage: none
source https://nuget.org/api/v2
nuget CNTK.CPUOnly
nuget FsLab
nuget MathNet.Numerics
nuget MathNet.Numerics.FSharp
nuget FSharp.Charting
nuget System.Core
"""

In [ ]:
Paket.Package ["CNTK.CPUOnly"; "FsLab"; "MathNet.Numerics"; "MathNet.Numerics.FSharp"; "FSharp.Charting"]

In [ ]:
#load "Paket.Generated.Refs.fsx"

In [ ]:
// All cntk dlls need to be copied in the same 
// folder with Cntk.Core.Managed-*.*.dll
#load "prepareWorkspace.fsx"

**After loading script and copying to /bin is complete, restart kernel before continuing.**

In [15]:
#r @"bin\Cntk.Core.Managed-2.6.dll"
#load "Paket.Generated.Refs.fsx"

let path = System.Environment.GetEnvironmentVariable("PATH")
let path' = sprintf "%s%c%s" path (System.IO.Path.PathSeparator) (System.IO.Path.GetFullPath("bin"))
System.Environment.SetEnvironmentVariable("PATH", path')

open CNTK
let cpu = DeviceDescriptor.UseDefaultDevice()
printfn "You are using CNTK for: %A" (cpu.Type)

You are using CNTK for: CPU


In [16]:
open System
open CNTK
open MathNet.Numerics
open MathNet.Numerics.Random
open MathNet.Numerics.Distributions;

let rand = System.Random(9)
let randInt max = seq { while true do yield rand.Next() % max }
let randn = Normal.Samples(rand, 0.0, 1.0)
let oneHotEncoding classCount classType =
    Array.init classCount (fun i -> if i = classType then 1.0 else 0.0)

let generateRandomDataSample sampleSize featureDim numClasses =
    // label vector
    let Y = randInt numClasses |> Seq.take sampleSize |> Array.ofSeq
    
    // feature vector
    let feature size = 
        randn 
        |> Seq.take sampleSize
        |> Seq.mapi (fun i num -> (num + 3.0) * float (Y.[i] + 1))
        |> Array.ofSeq    
    
    let X = Array.init featureDim (fun _ -> feature sampleSize)
    
    let Y_oneHot = Array.map (oneHotEncoding numClasses) Y
    
    Y_oneHot |> Array.transpose |> Array.append X |> Array.transpose

let inline shape (dims:int seq) : NDShape = NDShape.CreateNDShape dims
let parVec (pars:Parameter seq) = 
    let vector = new ParameterVector()
    pars |> Seq.iter (vector.Add)
    vector

let featureCount = 2
let labelCount = 2
let sampleCount = 32
let device = DeviceDescriptor.CPUDevice
let dataType = CNTK.DataType.Double
let feature = Variable.InputVariable(shape [|featureCount|], dataType, "Features")        
let initialization = CNTKLib.GlorotUniformInitializer()
let learningRate = 0.5
let index = System.Collections.Generic.Dictionary<string, obj>()

let linearLayer (inputVar : Variable) outputDim =
    let inputDim = inputVar.Shape.[0] 
    let weightParam = new Parameter(shape [inputDim;outputDim], dataType, initialization, device, "Weights")
    let biasParam = new Parameter(shape [outputDim], dataType, initialization, device, "Bias")
    
    index.Add("Weights", weightParam)
    index.Add("Bias", biasParam)
    
    let dotProduct =  CNTKLib.Times(inputVar, weightParam, "Weighted input")
    let layer = CNTKLib.Plus(new Variable(dotProduct), biasParam, "Layer")
    
    layer

In [17]:
let rows = generateRandomDataSample sampleCount featureCount labelCount
let z = linearLayer feature labelCount
let label = Variable.InputVariable(shape [labelCount], dataType, "output")
let loss = CNTKLib.CrossEntropyWithSoftmax(new Variable(z), label)
let evalError = CNTKLib.ClassificationError(new Variable(z), label)    
let lrSchedule = new CNTK.TrainingParameterScheduleDouble(learningRate, uint32 CNTK.DataUnit.Minibatch)
let learner = CNTKLib.SGDLearner(z.Parameters() |> parVec, lrSchedule)
let trainer = CNTK.Trainer.CreateTrainer(z, loss, evalError, ResizeArray<CNTK.Learner>([learner]))


# CNTK 101: Logistic Regression and ML Primer

## Intro

In [ ]:
let ImageUrl url width =
    sprintf "<img src=\"%s\" style=\"width: %dpx; height: auto\" alt=\"Could not load image, make sure url is correct\">" url width
    |> Util.Html
    |> Display

In [ ]:
// Figure 1
ImageUrl "https://www.cntk.ai/jup/cancer_data_plot.jpg" 400

In [ ]:
// Figure 2
ImageUrl "https://www.cntk.ai/jup/cancer_classify_plot.jpg" 400

In [ ]:
// Figure 3
ImageUrl "https://www.cntk.ai/jup/logistic_neuron.jpg" 300

In [2]:
let featureCount = 2
let labelCount = 2
let sampleCount = 32
let device = DeviceDescriptor.CPUDevice

## Data generation

In [3]:
open MathNet.Numerics
open MathNet.Numerics.Random
open MathNet.Numerics.Distributions;

let rand = System.Random(9)
let randInt max = seq { while true do yield rand.Next() % max }
let randn = Normal.Samples(rand, 0.0, 1.0)
let oneHotEncoding classCount classType =
    Array.init classCount (fun i -> if i = classType then 1.0 else 0.0)

let generateRandomDataSample sampleSize featureDim numClasses =
    // label vector
    let Y = randInt numClasses |> Seq.take sampleSize |> Array.ofSeq
    
    // feature vector
    let feature size = 
        randn 
        |> Seq.take sampleSize
        |> Seq.mapi (fun i num -> (num + 3.0) * float (Y.[i] + 1))
        |> Array.ofSeq    
    
    let X = Array.init featureDim (fun _ -> feature sampleSize)
    
    let Y_oneHot = Array.map (oneHotEncoding numClasses) Y
    
    Y_oneHot |> Array.transpose |> Array.append X |> Array.transpose
        
let rows = generateRandomDataSample sampleCount featureCount labelCount

rows.[0..5]

[|[|3.415654791; 3.675224115; 1.0; 0.0|]; [|2.494471585; 2.345477671; 1.0; 0.0|];
  [|0.4747809944; 3.076814035; 1.0; 0.0|];
  [|5.530792053; 4.132543366; 1.0; 0.0|]; [|2.581883888; 1.657196651; 1.0; 0.0|];
  [|1.614800661; 3.624784112; 1.0; 0.0|]|]

## Data visualization

In [ ]:
// Setup display support
#load "FSharp.Charting.fsx"

In [ ]:
open FSharp.Charting

let class1, class2 = rows |> Array.partition (fun row -> row |> Array.last |> (=)1.0)
let chartClass (cls:float[][]) = cls |> Array.map (fun x -> x.[0],x.[1]) |> Chart.Point

[class1;class2] 
|> List.map chartClass 
|> Chart.Combine 
|> fun chart -> 
    chart.WithXAxis(Title="Tumor size (in cm)")
         .WithYAxis(Title="Age (scaled)")

In [ ]:
// Setup display support
#load "XPlot.Plotly.fsx"

In [ ]:
open XPlot.Plotly

let inline color className = if className=1.0 then "Red" else "Blue"
let chartClassX (cls:float[][]) =
    let className = cls.[0] |> Array.last
    let points = cls |> Array.transpose
    Scatter(x = points.[0], y = points.[1], 
            mode = "markers", 
            marker = Marker(size=10, color=color className),
            name = className.ToString())

[chartClassX class1; chartClassX class2]
|> Chart.Plot
|> Chart.WithLayout (Layout(xaxis=Xaxis(title="Tumor size (in cm)"), yaxis=Yaxis(title="Age (scaled)")))
|> Chart.WithHeight 400
|> Chart.WithWidth 600

## Model Creation

In [ ]:
// Figure 4
ImageUrl "https://www.cntk.ai/jup/logistic_neuron2.jpg" 300

In [ ]:
"z=\sum_{i=1}^n w_i \\times x_i+b= \\textbf{w·x}+b" |> Util.Math

In [4]:
let inline shape (dims:int seq) : NDShape = NDShape.CreateNDShape dims
let dataType = CNTK.DataType.Double
let feature = Variable.InputVariable(shape [|featureCount|], dataType, "Features")

## Network setup

In [5]:
let initialization = CNTKLib.GlorotUniformInitializer()
let index = System.Collections.Generic.Dictionary<string, obj>()
let linearLayer (inputVar : Variable) outputDim =
    let inputDim = inputVar.Shape.[0] 
    let weightParam = new Parameter(shape [inputDim; outputDim], dataType, initialization, device, "Weights")
    let biasParam = new Parameter(shape [outputDim], dataType, initialization, device, "Bias")    
    
    index.Add("Weights", weightParam)
    index.Add("Bias", biasParam)
    
    let dotProduct =  CNTKLib.Times(inputVar, weightParam, "Weighted input")
    let layer = CNTKLib.Plus(new Variable(dotProduct), biasParam, "Layer")
    
    layer

let z = linearLayer feature labelCount

## Training

In [ ]:
"\\textbf{p}=softmax(z)" |> Util.Math

In [ ]:
"H(p)=-\sum_{j=1}^{|y|}y_j log(p_j)" |> Util.Math

In [6]:
let label = Variable.InputVariable(shape [labelCount], dataType, "output")
let loss = CNTKLib.CrossEntropyWithSoftmax(new Variable(z), label)

## Evaluation

In [7]:
let evalError = CNTKLib.ClassificationError(new Variable(z), label)

## Configure training

In [8]:
// Instantiate the trainer object to drive the model training
let parVec (pars:Parameter seq) = 
    let vector = new ParameterVector()
    pars |> Seq.iter (vector.Add)
    vector

let learningRate = 0.5
let lrSchedule = new CNTK.TrainingParameterScheduleDouble(learningRate, uint32 CNTK.DataUnit.Minibatch)

In [9]:
let learner = CNTKLib.SGDLearner(z.Parameters() |> parVec, lrSchedule)
let trainer = CNTK.Trainer.CreateTrainer(z, loss, evalError, ResizeArray<CNTK.Learner>([learner]))

Expression evaluation failed: External component has thrown an exception.
SEHExceptionExternal component has thrown an exception.
   at CNTK.CNTKLibPINVOKE.SGDLearner__SWIG_1(HandleRef jarg1, HandleRef jarg2)
   at CNTK.CNTKLib.SGDLearner(ParameterVector parameters, TrainingParameterScheduleDouble learningRateSchedule)
   at <StartupCode$FSI_0015>.$FSI_0015.main@() in C:\Users\arist.CTEAMTHES\source\repos\DS-AI\IfCntk\input.fsx:line 1
